In [1]:
import numpy as np
import pandas as pd
import re

import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df=pd.read_csv('energy_dataset.csv') # importing the energy dataset

In [3]:
df.head()

,time,generation biomass,generation fossil brown coal/lignite,generation fossil coal-derived gas,generation fossil gas,generation fossil hard coal,generation fossil oil,generation fossil oil shale,generation fossil peat,generation geothermal,...,generation waste,generation wind offshore,generation wind onshore,forecast solar day ahead,forecast wind offshore eday ahead,forecast wind onshore day ahead,total load forecast,total load actual,price day ahead,price actual
0,2015-01-01 00:00:00+01:00,447.0,329.0,0.0,4844.0,4821.0,162.0,0.0,0.0,0.0,...,196.0,0.0,6378.0,17.0,NaN,6436.0,26118.0,25385.0,50.10,65.41
1,2015-01-01 01:00:00+01:00,449.0,328.0,0.0,5196.0,4755.0,158.0,0.0,0.0,0.0,...,195.0,0.0,5890.0,16.0,NaN,5856.0,24934.0,24382.0,48.10,64.92
2,2015-01-01 02:00:00+01:00,448.0,323.0,0.0,4857.0,4581.0,157.0,0.0,0.0,0.0,...,196.0,0.0,5461.0,8.0,NaN,5454.0,23515.0,22734.0,47.33,64.48
3,2015-01-01 03:00:00+01:00,438.0,254.0,0.0,4314.0,4131.0,160.0,0.0,0.0,0.0,...,191.0,0.0,5238.0,2.0,NaN,5151.0,22642.0,21286.0,42.27,59.32
4,2015-01-01 04:00:00+01:00,428.0,187.0,0.0,4130.0,3840.0,156.0,0.0,0.0,0.0,...,189.0,0.0,4935.0,9.0,NaN,4861.0,21785.0,20264.0,38.41,56.04


In [4]:
df.info() #Displaying informations about the dataset.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35064 entries, 0 to 35063
Data columns (total 29 columns):
 #   Column                                       Non-Null Count  Dtype  
---  ------                                       --------------  -----  
 0   time                                         35064 non-null  object 
 1   generation biomass                           35045 non-null  float64
 2   generation fossil brown coal/lignite         35046 non-null  float64
 3   generation fossil coal-derived gas           35046 non-null  float64
 4   generation fossil gas                        35046 non-null  float64
 5   generation fossil hard coal                  35046 non-null  float64
 6   generation fossil oil                        35045 non-null  float64
 7   generation fossil oil shale                  35046 non-null  float64
 8   generation fossil peat                       35046 non-null  float64
 9   generation geothermal                        35046 non-null  float64
 10

In [5]:
df.shape #shape of the dataframe

(35064, 29)

In [6]:
#dropping columns with no non-null values.
df=df.drop(['forecast wind offshore eday ahead','generation hydro pumped storage aggregated'],axis=1) 
df.isnull().sum() #counting null values in each column.

time                                            0
generation biomass                             19
generation fossil brown coal/lignite           18
generation fossil coal-derived gas             18
generation fossil gas                          18
generation fossil hard coal                    18
generation fossil oil                          19
generation fossil oil shale                    18
generation fossil peat                         18
generation geothermal                          18
generation hydro pumped storage consumption    19
generation hydro run-of-river and poundage     19
generation hydro water reservoir               18
generation marine                              19
generation nuclear                             17
generation other                               18
generation other renewable                     18
generation solar                               18
generation waste                               19
generation wind offshore                       18


In [7]:
#using linear interpolation to fill in missing values in every column.
for col in df.isnull().sum().index: 
    if df.isnull().sum()[col]>0:
        df[col]=df[col].interpolate(method='linear')

In [8]:
df.isnull().sum() #No missing values 

time                                           0
generation biomass                             0
generation fossil brown coal/lignite           0
generation fossil coal-derived gas             0
generation fossil gas                          0
generation fossil hard coal                    0
generation fossil oil                          0
generation fossil oil shale                    0
generation fossil peat                         0
generation geothermal                          0
generation hydro pumped storage consumption    0
generation hydro run-of-river and poundage     0
generation hydro water reservoir               0
generation marine                              0
generation nuclear                             0
generation other                               0
generation other renewable                     0
generation solar                               0
generation waste                               0
generation wind offshore                       0
generation wind onsh

In [9]:
#the dataframe represents the hourly generation of each energy source.
#grouping by day instead of hours aggregating power generation by thr sum.
df['time']=pd.to_datetime(df['time'],utc=True).dt.date 
df_agg=df.groupby(['time']).sum()                       
df_agg.drop(['price day ahead', 'price actual'],axis=1,inplace=True)
df_agg['Date']=df_agg.index
df_agg.index=np.arange(0,df_agg.shape[0])
df_agg=df_agg[['Date','generation biomass', 'generation fossil brown coal/lignite',
       'generation fossil coal-derived gas', 'generation fossil gas',
       'generation fossil hard coal', 'generation fossil oil',
       'generation fossil oil shale', 'generation fossil peat',
       'generation geothermal', 'generation hydro pumped storage consumption',
       'generation hydro run-of-river and poundage',
       'generation hydro water reservoir', 'generation marine',
       'generation nuclear', 'generation other', 'generation other renewable',
       'generation solar', 'generation waste', 'generation wind offshore',
       'generation wind onshore', 'forecast solar day ahead',
       'forecast wind onshore day ahead', 'total load forecast',
       'total load actual']]
df_agg.head()

,Date,generation biomass,generation fossil brown coal/lignite,generation fossil coal-derived gas,generation fossil gas,generation fossil hard coal,generation fossil oil,generation fossil oil shale,generation fossil peat,generation geothermal,...,generation other,generation other renewable,generation solar,generation waste,generation wind offshore,generation wind onshore,forecast solar day ahead,forecast wind onshore day ahead,total load forecast,total load actual
0,2014-12-31,447.0,329.0,0.0,4844.0,4821.0,162.0,0.0,0.0,0.0,...,43.0,73.0,49.0,196.0,0.0,6378.0,17.0,6436.0,26118.0,25385.0
1,2015-01-01,10205.0,5849.0,0.0,100507.0,98387.0,3876.0,0.0,0.0,0.0,...,1016.0,1764.0,27448.0,4825.0,0.0,95856.0,25188.0,94618.0,594078.0,575207.0
2,2015-01-02,9357.0,0.0,0.0,85588.0,30195.0,4820.0,0.0,0.0,0.0,...,1095.0,1487.0,30061.0,4478.0,0.0,265441.0,28493.0,266809.0,660466.0,652525.0
3,2015-01-03,10485.0,0.0,0.0,82843.0,33023.0,5515.0,0.0,0.0,0.0,...,1876.0,1580.0,27501.0,4856.0,0.0,219294.0,27201.0,218722.0,612020.0,602346.0
4,2015-01-04,9513.0,326.0,0.0,84627.0,55567.0,8559.0,0.0,0.0,0.0,...,1969.0,1393.0,57659.0,3819.0,0.0,192387.0,55009.0,192548.0,652029.0,650518.0


In [10]:
df_price=df[['time','price day ahead','price actual']] 
df_price=df_price.groupby(['time']).mean()
df_price['Date']=df_price.index
df_price.index=np.arange(0,df_price.shape[0])                #Aggregating the price by the average in a seperate dataframe
df_price=df_price[['Date','price day ahead','price actual']]

df_agg=df_agg.merge(df_price,how='outer',on='Date').round(2) #merging the price and the previous dataframe into one.
df_agg.head()

,Date,generation biomass,generation fossil brown coal/lignite,generation fossil coal-derived gas,generation fossil gas,generation fossil hard coal,generation fossil oil,generation fossil oil shale,generation fossil peat,generation geothermal,...,generation solar,generation waste,generation wind offshore,generation wind onshore,forecast solar day ahead,forecast wind onshore day ahead,total load forecast,total load actual,price day ahead,price actual
0,2014-12-31,447.0,329.0,0.0,4844.0,4821.0,162.0,0.0,0.0,0.0,...,49.0,196.0,0.0,6378.0,17.0,6436.0,26118.0,25385.0,50.10,65.41
1,2015-01-01,10205.0,5849.0,0.0,100507.0,98387.0,3876.0,0.0,0.0,0.0,...,27448.0,4825.0,0.0,95856.0,25188.0,94618.0,594078.0,575207.0,45.03,62.09
2,2015-01-02,9357.0,0.0,0.0,85588.0,30195.0,4820.0,0.0,0.0,0.0,...,30061.0,4478.0,0.0,265441.0,28493.0,266809.0,660466.0,652525.0,17.60,69.44
3,2015-01-03,10485.0,0.0,0.0,82843.0,33023.0,5515.0,0.0,0.0,0.0,...,27501.0,4856.0,0.0,219294.0,27201.0,218722.0,612020.0,602346.0,25.34,65.22
4,2015-01-04,9513.0,326.0,0.0,84627.0,55567.0,8559.0,0.0,0.0,0.0,...,57659.0,3819.0,0.0,192387.0,55009.0,192548.0,652029.0,650518.0,30.66,58.91


In [11]:
#Getting 2 different dataframes for renewables and non-renewables seperately which will facilitate our on coming calculations.

df_non_renewable=df_agg[['generation fossil brown coal/lignite',
       'generation fossil coal-derived gas', 'generation fossil gas',
       'generation fossil hard coal','generation nuclear', 'generation fossil oil',
       'generation fossil oil shale', 'generation fossil peat']]
df_renewable=df_agg[['generation biomass','generation geothermal', 'generation hydro pumped storage consumption',
       'generation hydro run-of-river and poundage',
       'generation hydro water reservoir', 'generation marine',
        'generation other renewable',
       'generation solar', 'generation waste', 'generation wind offshore',
       'generation wind onshore']]


In [12]:
df_renewable

,generation biomass,generation geothermal,generation hydro pumped storage consumption,generation hydro run-of-river and poundage,generation hydro water reservoir,generation marine,generation other renewable,generation solar,generation waste,generation wind offshore,generation wind onshore
0,447.0,0.0,863.0,1051.0,1899.0,0.0,73.0,49.0,196.0,0.0,6378.0
1,10205.0,0.0,26117.0,25366.0,53977.0,0.0,1764.0,27448.0,4825.0,0.0,95856.0
2,9357.0,0.0,36260.0,31200.0,89007.0,0.0,1487.0,30061.0,4478.0,0.0,265441.0
3,10485.0,0.0,32666.0,33073.0,93538.0,0.0,1580.0,27501.0,4856.0,0.0,219294.0
4,9513.0,0.0,13765.0,35566.0,77887.0,0.0,1393.0,57659.0,3819.0,0.0,192387.0
...,...,...,...,...,...,...,...,...,...,...,...
1457,7312.0,0.0,3248.0,28498.0,90356.0,0.0,2210.0,16157.0,7141.0,0.0,26782.0
1458,7591.0,0.0,2098.0,27648.0,59279.0,0.0,2346.0,13224.0,7364.0,0.0,98670.0
1459,6924.0,0.0,14685.0,26811.0,43856.0,0.0,2412.0,18126.0,7458.0,0.0,171767.0
1460,6608.0,0.0,14512.0,25592.0,47100.0,0.0,2364.0,25271.0,7470.0,0.0,118190.0


In [13]:
def day_total(df):                  #Function to caluclate the daily total generated power.
    tot=[]
    for i in range(0,df.shape[0]):
        total=df.iloc[i].sum()
        tot.append(total)
     
    return tot
    


In [14]:
df_agg['renewable generation']=day_total(df_renewable)        #Calculating the total generated power of both renewables and non-renewables seperately.
df_agg['non renewable generation']=day_total(df_non_renewable) #Creating the 2 columns
df_agg['Total generation']=df_agg['renewable generation']+df_agg['non renewable generation']
df_agg.head()

,Date,generation biomass,generation fossil brown coal/lignite,generation fossil coal-derived gas,generation fossil gas,generation fossil hard coal,generation fossil oil,generation fossil oil shale,generation fossil peat,generation geothermal,...,generation wind onshore,forecast solar day ahead,forecast wind onshore day ahead,total load forecast,total load actual,price day ahead,price actual,renewable generation,non renewable generation,Total generation
0,2014-12-31,447.0,329.0,0.0,4844.0,4821.0,162.0,0.0,0.0,0.0,...,6378.0,17.0,6436.0,26118.0,25385.0,50.10,65.41,10956.0,17252.0,28208.0
1,2015-01-01,10205.0,5849.0,0.0,100507.0,98387.0,3876.0,0.0,0.0,0.0,...,95856.0,25188.0,94618.0,594078.0,575207.0,45.03,62.09,245558.0,378957.0,624515.0
2,2015-01-02,9357.0,0.0,0.0,85588.0,30195.0,4820.0,0.0,0.0,0.0,...,265441.0,28493.0,266809.0,660466.0,652525.0,17.60,69.44,467291.0,291133.0,758424.0
3,2015-01-03,10485.0,0.0,0.0,82843.0,33023.0,5515.0,0.0,0.0,0.0,...,219294.0,27201.0,218722.0,612020.0,602346.0,25.34,65.22,422993.0,291660.0,714653.0
4,2015-01-04,9513.0,326.0,0.0,84627.0,55567.0,8559.0,0.0,0.0,0.0,...,192387.0,55009.0,192548.0,652029.0,650518.0,30.66,58.91,391989.0,316036.0,708025.0


In [15]:
df_agg['Date']=pd.to_datetime(df_agg['Date'])    #Casting the date into datetime data type
per=df_agg['Date'].dt.to_period('Y')             
df_year=df_agg.groupby(per).mean()                #grouping by year with a mean aggregation
df_year

,generation biomass,generation fossil brown coal/lignite,generation fossil coal-derived gas,generation fossil gas,generation fossil hard coal,generation fossil oil,generation fossil oil shale,generation fossil peat,generation geothermal,generation hydro pumped storage consumption,...,generation wind onshore,forecast solar day ahead,forecast wind onshore day ahead,total load forecast,total load actual,price day ahead,price actual,renewable generation,non renewable generation,Total generation
Date,,,,,,,,,,,,,,,,,,,,,
2014,447.000000,329.000000,0.0,4844.000000,4821.000000,162.000000,0.0,0.0,0.0,863.000000,...,6378.000000,17.000000,6436.000000,26118.000000,25385.000000,50.100000,65.410000,10956.000000,17252.000000,28208.000000
2015,11791.034247,12352.613699,0.0,121385.347945,128344.046575,7927.913699,0.0,0.0,0.0,12796.295890,...,131485.917808,34852.465753,131403.391781,681718.652055,680698.520548,50.324740,61.359808,279849.905479,419943.182192,699793.087671
2016,8771.672131,9138.163934,0.0,123419.954918,87685.263661,6867.146175,0.0,0.0,0.0,13581.042350,...,129975.229508,33913.128415,130057.327869,684358.601093,684188.374317,39.668443,47.438361,299918.628415,380186.933060,680105.561475
2017,8171.230137,13060.816438,0.0,155363.490411,104004.975342,7073.616438,0.0,0.0,0.0,10043.767123,...,129152.200000,35903.547945,129235.879452,692657.786301,692600.706849,52.233288,59.322082,249594.791781,431810.019178,681404.810959
2018,8085.034247,8469.010959,0.0,139628.772603,88619.156164,6772.554795,0.0,0.0,0.0,9226.532877,...,134010.546575,33482.928767,134526.000000,697570.846575,697490.384932,57.300247,63.444548,292765.001370,389440.135616,682205.136986


In [16]:
df_agg.head()

,Date,generation biomass,generation fossil brown coal/lignite,generation fossil coal-derived gas,generation fossil gas,generation fossil hard coal,generation fossil oil,generation fossil oil shale,generation fossil peat,generation geothermal,...,generation wind onshore,forecast solar day ahead,forecast wind onshore day ahead,total load forecast,total load actual,price day ahead,price actual,renewable generation,non renewable generation,Total generation
0,2014-12-31,447.0,329.0,0.0,4844.0,4821.0,162.0,0.0,0.0,0.0,...,6378.0,17.0,6436.0,26118.0,25385.0,50.10,65.41,10956.0,17252.0,28208.0
1,2015-01-01,10205.0,5849.0,0.0,100507.0,98387.0,3876.0,0.0,0.0,0.0,...,95856.0,25188.0,94618.0,594078.0,575207.0,45.03,62.09,245558.0,378957.0,624515.0
2,2015-01-02,9357.0,0.0,0.0,85588.0,30195.0,4820.0,0.0,0.0,0.0,...,265441.0,28493.0,266809.0,660466.0,652525.0,17.60,69.44,467291.0,291133.0,758424.0
3,2015-01-03,10485.0,0.0,0.0,82843.0,33023.0,5515.0,0.0,0.0,0.0,...,219294.0,27201.0,218722.0,612020.0,602346.0,25.34,65.22,422993.0,291660.0,714653.0
4,2015-01-04,9513.0,326.0,0.0,84627.0,55567.0,8559.0,0.0,0.0,0.0,...,192387.0,55009.0,192548.0,652029.0,650518.0,30.66,58.91,391989.0,316036.0,708025.0


In [22]:
#Function which determines the most generating energy source and returns a list for every observation.

def maximumgen(df):
    max_gen=[]
    for i in range(0,len(df)):
        maximum=df.loc[i].values.argmax()
        max_gen.append(df.loc[i].index[maximum])
    return max_gen
    

        


In [23]:
df_renewable['most generating renewable']=maximumgen(df_renewable)
df_non_renewable['most generating non-renewable']=maximumgen(df_non_renewable)
df_agg['most generating renewable']=df_renewable['most generating renewable']
df_agg['most generating non-renewable']=df_non_renewable['most generating non-renewable']

<ipython-input-23-908bdeaf1346>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_renewable['most generating renewable']=maximumgen(df_renewable)
<ipython-input-23-908bdeaf1346>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_non_renewable['most generating non-renewable']=maximumgen(df_non_renewable)


In [24]:
df_renewable['most generating renewable'].value_counts()

generation wind onshore             1205
generation hydro water reservoir     241
generation solar                      16
Name: most generating renewable, dtype: int64

In [25]:
df_non_renewable['most generating non-renewable'].value_counts()

generation nuclear             917
generation fossil gas          397
generation fossil hard coal    148
Name: most generating non-renewable, dtype: int64

In [26]:
df_agg.head()

,Date,generation biomass,generation fossil brown coal/lignite,generation fossil coal-derived gas,generation fossil gas,generation fossil hard coal,generation fossil oil,generation fossil oil shale,generation fossil peat,generation geothermal,...,forecast wind onshore day ahead,total load forecast,total load actual,price day ahead,price actual,renewable generation,non renewable generation,Total generation,most generating renewable,most generating non-renewable
0,2014-12-31,447.0,329.0,0.0,4844.0,4821.0,162.0,0.0,0.0,0.0,...,6436.0,26118.0,25385.0,50.10,65.41,10956.0,17252.0,28208.0,generation wind onshore,generation nuclear
1,2015-01-01,10205.0,5849.0,0.0,100507.0,98387.0,3876.0,0.0,0.0,0.0,...,94618.0,594078.0,575207.0,45.03,62.09,245558.0,378957.0,624515.0,generation wind onshore,generation nuclear
2,2015-01-02,9357.0,0.0,0.0,85588.0,30195.0,4820.0,0.0,0.0,0.0,...,266809.0,660466.0,652525.0,17.60,69.44,467291.0,291133.0,758424.0,generation wind onshore,generation nuclear
3,2015-01-03,10485.0,0.0,0.0,82843.0,33023.0,5515.0,0.0,0.0,0.0,...,218722.0,612020.0,602346.0,25.34,65.22,422993.0,291660.0,714653.0,generation wind onshore,generation nuclear
4,2015-01-04,9513.0,326.0,0.0,84627.0,55567.0,8559.0,0.0,0.0,0.0,...,192548.0,652029.0,650518.0,30.66,58.91,391989.0,316036.0,708025.0,generation wind onshore,generation nuclear


In [27]:
renew_agg=df_agg.groupby(['most generating renewable']).count()[['generation biomass']]
non_renew_agg=df_agg.groupby(['most generating non-renewable']).count()[['generation biomass']]
count=pd.concat([renew_agg,non_renew_agg])

In [28]:
count.columns=['Number of days as maximum output']
count.index

Index(['generation hydro water reservoir', 'generation solar',
       'generation wind onshore', 'generation fossil gas',
       'generation fossil hard coal', 'generation nuclear'],
      dtype='object')

In [29]:
count

,Number of days as maximum output
generation hydro water reservoir,241
generation solar,16
generation wind onshore,1205
generation fossil gas,397
generation fossil hard coal,148
generation nuclear,917


In [30]:
df_agg['generation wind onshore'].sum()

191624074.5

In [31]:
gen_average=[]
gen_ratio=[]
total_mean=df_agg['Total generation'].mean()

for val in count.index:
    gen_average.append(df_agg[val].mean())
    gen_ratio.append(df_agg[val].mean()/total_mean)

count['daily average generation']=gen_average
count['average daily generation ratio']=gen_ratio
count['Most generating sources']=count.index
count=count.reset_index(drop=True)
count=count[['Most generating sources','Number of days as maximum output', 'daily average generation',
       'average daily generation ratio']]   
    

In [32]:
count.to_excel('generation.xlsx')